In [2]:
from pyrosetta import *
from pyrosetta.rosetta import *
init()

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.11+release.7c7790f-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.11+release.7c7790f7c7790fbb898f8210b54513b9f9c8a927c3e8d7f 2018-03-14T13:19:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r173 2018.11+release.7c7790f 7c7790fbb898f8210b54513b9f9c8a927c3e8d7f http://www.pyrosetta.org 2018-03-14T13:19:04
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.11+release.7c7790f-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=209234815 seed_offset=0 real_seed=209234815
core.init.random: RandomGenerator:i

In [3]:
import math
import random
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from pyrosetta.rosetta.protocols.moves import *
from pyrosetta.rosetta.core.id import *

In [4]:
def within_360(ang):
    
    return ang - 360.0*int(ang/360.0)

In [5]:
def numpy_hessian(x, h):
        
    x_grad = np.gradient(x, h)
    hessian = np.empty((x.ndim, x.ndim) + x.shape, dtype = x.dtype)
    
    for k, grad_k in enumerate(x_grad):
        tmp_grad = np.gradient(grad_k, h)
        for l, grad_kl in enumerate(tmp_grad):
            hessian[k, l, :, :] = grad_kl
            
    return hessian

In [6]:
def initialize_pose(seq1, seq2):
    
    n_residues = len(seq1) + len(seq2)
    
    dof_dict = {}
    
    pose = pyrosetta.rosetta.protocols.recces.pose_setup_turner(seq1, seq2)
    movemap = MoveMap()
    
    for ii in range(n_residues-1):
    
        dof_dict.update({6*ii:TorsionID(ii+2, BB, 1), 6*ii+1:TorsionID(ii+2, BB, 2), 6*ii+2:TorsionID(ii+2, BB, 3),
                 6*ii+3:TorsionID(ii+1, BB, 5), 6*ii+4:TorsionID(ii+1, BB, 6)})
        
        movemap.set(TorsionID(ii+2, BB, 1), True)
        movemap.set(TorsionID(ii+2, BB, 2), True)
        movemap.set(TorsionID(ii+2, BB, 3), True)
        movemap.set(TorsionID(ii+1, BB, 5), True)
        movemap.set(TorsionID(ii+1, BB, 6), True)
        
    for ii in range(n_residues-1):
        
        dof_dict.update({6*ii+5:TorsionID(ii+1, CHI, 1)})
        
    dof_dict.update({6*(n_residues-1):TorsionID(n_residues, CHI, 1)})
    
    movemap.set_chi(True)
    
    return pose, movemap, dof_dict

In [7]:
def find_minimum(pose, sf, movemap):
    
    
    minimum = Pose()
    minimum.assign(pose)
    #observer = AddPyMOLObserver(minimum, True)
    sf(minimum)
    n_residues = pose.total_residue()
    
    movemap = MoveMap()
    for ii in range(n_residues-1):
        
        minimum.set_alpha(  ii+2, random.uniform(0, 360))
        minimum.set_beta(   ii+2, random.uniform(0, 360))
        minimum.set_gamma(  ii+2, random.uniform(0, 360))
        minimum.set_epsilon(ii+1, random.uniform(0, 360))
        minimum.set_zeta(   ii+1, random.uniform(0, 360))
             
    for ii in range(n_residues):
        
        pose.set_chi(ii+1, random.uniform(0, 360))
    '''
    minmap = rosetta.core.optimization.MinimizerMap()
    minmap.setup(pose, movemap)
    sf.setup_for_minimizing(pose, minmap)
    multifunc = rosetta.core.optimization.AtomTreeMultifunc(pose, minmap, sf)
    num_dofs = minmap.nangles()
    print num_dofs
    '''
    minmover = rosetta.protocols.minimization_packing.MinMover(movemap, sf, 'dfpmin', 1e-5, True)
    
    minmover.apply(minimum)
    
    return minimum   

In [13]:
pose, movemap, dof_dict = initialize_pose('aa', 'uu')
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('rna/denovo/rna_hires')

In [14]:
#pose.assign(find_minimum(pose, scorefxn, movemap))
minmover = rosetta.protocols.minimization_packing.MinMover(movemap, scorefxn, 'dfpmin', 1e-5, True)
pmm = PyMOLMover()
pmm.apply(pose)
print dof_dict
#minmover.apply(pose)
pmm.apply(pose)

{0: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587b20>, 1: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587b90>, 2: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587c38>, 3: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587c70>, 4: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587bc8>, 5: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587df8>, 6: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587c00>, 7: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587ae8>, 8: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587810>, 9: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587880>, 10: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587ca8>, 11: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587e30>, 12: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587ce0>, 13: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46ec587d18>, 14: <pyrosetta.rosetta.core.id.TorsionID object at 0x7f46

In [18]:
import itertools

trial = Pose()
trial.assign(pose)

pose.set_alpha(2, -64)
pose.set_beta(2, 176)
pose.set_gamma(2, 53)
pose.set_epsilon(1, -150)
pose.set_zeta(1, -79)
pose.set_chi(1, 79)
pose.set_chi(2, 79)

pose.set_alpha(4, -64)
pose.set_beta(4, 176)
pose.set_gamma(4, 53)
pose.set_epsilon(3, -150)
pose.set_zeta(3, -79)
pose.set_chi(3, 79)

pose.set_chi(4, 79)
pmm.apply(pose)
h = 0.5
h_rad = h*np.pi/180

dofs = []
dofs.append(pose.alpha(2))
dofs.append(pose.beta(2))
dofs.append(pose.gamma(2))
dofs.append(pose.epsilon(1))
dofs.append(pose.zeta(1))
dofs.append(pose.chi(1))

dofs.append(pose.alpha(3))
dofs.append(pose.beta(3))
dofs.append(pose.gamma(3))
dofs.append(pose.epsilon(2))
dofs.append(pose.zeta(2))
dofs.append(pose.chi(2))

dofs.append(pose.alpha(4))
dofs.append(pose.beta(4))
dofs.append(pose.gamma(4))
dofs.append(pose.epsilon(3))
dofs.append(pose.zeta(3))
dofs.append(pose.chi(3))
dofs.append(pose.chi(4))

print dofs
hessian = np.zeros((len(dofs), len(dofs)))

n_pts = int(4/h + 1)
ind = int(n_pts/2)

tor_ranges = np.zeros((n_pts, len(dofs)))
energy = np.zeros((n_pts, n_pts))

for ii, dof in enumerate(dofs):
            
    tor_ranges[:, ii] = dof + np.arange(-2, 2+h, h)
    
for pair in list(itertools.combinations(range(len(dofs)), 2)):
    
    x = tor_ranges[:, pair[0]]
    y = tor_ranges[:, pair[1]]

    for ii, xx in enumerate(x):
        
        if ii in [6,7,8,9,10,11]:
        
            continue
            
        pose.set_torsion(dof_dict[pair[0]], xx)
        
        for jj, yy in enumerate(y):
       
            if ii in [6,7,8,9,10,11]:
        
                continue
            
            pose.set_torsion(dof_dict[pair[1]], yy)
            
            energy[ii, jj] = scorefxn(pose)

            pose.set_torsion(dof_dict[pair[1]], dofs[pair[1]])
            
        pose.set_torsion(dof_dict[pair[0]], dofs[pair[0]])
        
    hess = numpy_hessian(energy, h)
    
    d2E_dx2 =  hess[0,0,:,:]
    d2E_dy2 =  hess[1,1,:,:]
    d2E_dxdy = hess[0,1,:,:]
    
    if hessian[pair[0], pair[0]] == 0:
        
        hessian[pair[0], pair[0]] = d2E_dx2[5,5]
        
    if hessian[pair[1], pair[1]] == 0:
        
        hessian[pair[1], [pair[1]]] = d2E_dy2[ind, ind]
        
    hessian[pair[0],pair[1]] = d2E_dxdy[ind, ind]
    hessian[pair[1],pair[0]] = d2E_dxdy[ind, ind]
    
               
print dofs                    
pmm.apply(pose)
    

[-64.0, 176.0, 53.0, -150.0, -79.0, 79.0, 176.0, 53.0, -150.0, -79.0, 79.0, 79.0, -64.0, 176.0, 53.0, -150.0, -79.0, 79.0, 79.0]
[-64.0, 176.0, 53.0, -150.0, -79.0, 79.0, 176.0, 53.0, -150.0, -79.0, 79.0, 79.0, -64.0, 176.0, 53.0, -150.0, -79.0, 79.0, 79.0]


In [19]:
import scipy
import scipy.linalg

w,v = scipy.linalg.eigh(hessian)

In [20]:
print w, v
pmm.apply(pose)

[ -2.62523118e-02  -1.49241193e-02  -9.01142921e-03  -1.04039648e-03
  -7.01461081e-04  -1.39485797e-05   6.22487965e-04   9.68105644e-04
   1.85945097e-03   8.76376065e-03   1.10057882e-02   2.13421991e-02
   3.35328868e-01   1.07832454e+01   1.09589956e+01   1.09589956e+01
   1.09589956e+01   1.09589956e+01   1.09735698e+01] [[  7.50931158e-04  -8.70953760e-05   2.84511183e-05  -3.48650062e-06
   -8.87244826e-06   2.19473550e-06   1.71672620e-05   4.39521289e-05
    2.93365709e-05   4.10780704e-05   3.45473609e-04   4.97984684e-04
    1.72797906e-03   9.99998035e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.71050259e-46]
 [  1.97006273e-01  -3.94179168e-01   3.66006185e-03   1.33049476e-01
   -3.73539073e-02   1.12910403e-01  -4.38163644e-01  -1.35075003e-01
    5.66959265e-01   1.60814951e-01  -3.51729774e-01  -2.79977165e-01
   -1.19639727e-01   2.75404657e-04   1.03595727e-17  -2.27380250e-17
   -2.92987911e-17   8.99662622e-18   6.09786536e-17]
 [ 

In [40]:
K = w[10]
mode = v[:,10]
print mode
observer = AddPyMOLObserver(pose, True)
h = np.arange(-15,15,1)
h = np.concatenate((np.append(h, 15), h[: :-1]))
print h

for hh in h:
    
    dofs_new = dofs+hh*mode
    
    for jj, dof in enumerate(dofs_new):
        
        pose.set_torsion(dof_dict[jj], dof)
        
    scorefxn(pose)
        


[  3.45473609e-04  -3.51729774e-01  -5.72165070e-01  -8.72599326e-02
   1.30091358e-01   1.45641128e-01   0.00000000e+00  -1.38777878e-17
   0.00000000e+00  -3.46944695e-18  -4.33680869e-19  -5.41322349e-01
  -2.11633650e-02   4.59549877e-02   4.18591624e-02   6.52782874e-02
  -3.04587159e-01  -1.07094860e-01   3.11958425e-01]
[-15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2
   3   4   5   6   7   8   9  10  11  12  13  14  15  14  13  12  11  10
   9   8   7   6   5   4   3   2   1   0  -1  -2  -3  -4  -5  -6  -7  -8
  -9 -10 -11 -12 -13 -14 -15]


In [26]:
print h


[-15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2
   3   4   5   6   7   8   9  10  11  12  13  14]


In [28]:
h.concatenate(np.arange(15,-15,1))

AttributeError: 'numpy.ndarray' object has no attribute 'concatenate'

In [35]:
np.concatenate((h, h[ : :-1]))

array([-15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,
        -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        11,  12,  13,  14,  14,  13,  12,  11,  10,   9,   8,   7,   6,
         5,   4,   3,   2,   1,   0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,
        -8,  -9, -10, -11, -12, -13, -14, -15])

In [36]:
np.append(h, 15)

array([-15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,
        -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        11,  12,  13,  14,  15])

In [37]:
print h


[-15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2
   3   4   5   6   7   8   9  10  11  12  13  14]
